In [2]:
import requests
import pandas as pd
import csv

In [3]:
#!conda install beautifulsoup4
from bs4 import BeautifulSoup

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')
file = open('postal_codes_toronto.csv', 'w')
csv_wr = csv.writer(file)
csv_wr.writerow(['PostalCode', 'Borough', 'Neighbourhood'])

table = soup.find('table', class_ = 'wikitable')
table_rows = table.find_all('tr')
postal_codes = []
boroughs = [] 
neighbourhoods = [] 

for row in table_rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  
            
            postalcode = columns[0].text.split('\n')[0]
            postal_codes.append(postalcode)
            borough = columns[1].text.split('\n')[0]
            boroughs.append(borough)
            neighbourhood = columns[2].text.split('\n')[0]    
            neighbourhoods.append(neighbourhood)
            
    except Exception as e : 
        pass    
    
    
postal_present = []
for index_i, postal_i in enumerate(postal_codes) :   
    if postal_i not in postal_present :
        nbds = neighbourhoods[index_i]
        for index_f, postal_f in enumerate(postal_codes) :
            if postal_i == postal_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_wr.writerow([postal_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postal_present.append(postal_i)
file.close()

In [5]:
df = pd.read_csv('postal_codes_toronto.csv')

In [6]:
df2 = df.dropna()
df2.reset_index(drop=True, inplace=True)

In [7]:
df2['Neighbourhood'] = df2.Neighbourhood.str.replace('/',',')
df2 = df2.astype(str)
df2.to_csv('Toronto_neighbourhoods.csv')

df2.shape

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(103, 3)

In [9]:
df_loc = pd.read_csv("http://cocl.us/Geospatial_data")

In [10]:
df_loc.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [11]:
df_nbd = pd.read_csv('Toronto_neighbourhoods.csv',index_col=[0])
df_nbd.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [13]:
df_final = pd.merge(df_nbd,df_loc,how="inner")
df_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [14]:
df_final.to_csv('Latitude_longitude_data.csv')

# Neighborhood Clustering

# Creating a map of Toronto

In [16]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [18]:
!conda install -c conda-forge geopy --yes

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt



In [20]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [21]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
df_nbd = pd.read_csv('Latitude_longitude_data.csv')
# add markers to map
for lat, lng, borough, neighbourhood in zip(df_nbd['Latitude'], df_nbd['Longitude'], df_nbd['Borough'], df_nbd['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Exploring Central Toronto

In [22]:
central_toronto_data = df_nbd[df_nbd['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto_data.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,62,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,67,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [23]:
address = 'Central Toronto'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


# Creating a map of Central Toronto

In [24]:
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto_data['Latitude'], central_toronto_data['Longitude'], central_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

# FourSquare Credentials

In [33]:
CLIENT_ID = 'OTDONZY24DU4MH4W3EZWYUKVRSIEBOE1JXGL5ZF4AMNZZHDK' # your Foursquare ID
CLIENT_SECRET = 'MX2UQNEYPBAZSZAYTM2KL0MUTUGL5KN5AS4AKLIJQ5JFKLEI' # your Foursquare Secret
VERSION = '20200405' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OTDONZY24DU4MH4W3EZWYUKVRSIEBOE1JXGL5ZF4AMNZZHDK
CLIENT_SECRET:MX2UQNEYPBAZSZAYTM2KL0MUTUGL5KN5AS4AKLIJQ5JFKLEI


# Exploring the First Neighborhood

In [34]:
central_toronto_data.loc[0, 'Neighbourhood']

'Lawrence Park'

In [35]:
neighbourhood_latitude = central_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = central_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = central_toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


# Top 100 Venues around Lawrence Park

In [36]:
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)


# For all Neighborhoods 

In [37]:
import json
from pandas.io.json import json_normalize

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
central_toronto_venues = getNearbyVenues(names=central_toronto_data['Neighbourhood'],
                                   latitudes=central_toronto_data['Latitude'],
                                   longitudes=central_toronto_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex , North Midtown , Yorkville
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park


In [40]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(110, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Roselawn,43.711695,-79.416936,Aquatics Academy Inc.,43.709951,-79.412127,Pool


In [41]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,38,38,38,38,38,38
Davisville North,6,6,6,6,6,6
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park , Summerhill East",1,1,1,1,1,1
North Toronto West,19,19,19,19,19,19
Roselawn,2,2,2,2,2,2
"Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park",15,15,15,15,15,15
"The Annex , North Midtown , Yorkville",22,22,22,22,22,22


In [42]:
print('There are {} unique categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 62 unique categories.


# Analysis for all Neighborhoods

In [43]:
# one hot encoding
c_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
c_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [c_toronto_onehot.columns[-1]] + list(c_toronto_onehot.columns[:-1])
c_toronto_onehot = c_toronto_onehot[fixed_columns]

c_toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Playground,Pool,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
c_toronto_grouped = c_toronto_onehot.groupby('Neighborhood').mean().reset_index()
c_toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Playground,Pool,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.026316,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.026316,0.000000,0.078947,0.026316,0.026316,0.000000,0.026316,0.000000,0.000000,0.000000,0.0,0.026316,0.000000,0.026316,0.026316,0.052632,0.000000,0.000000,0.000000,0.026316,0.052632,0.00,0.000000,0.000000,0.000000,0.000000,0.026316,0.026316,0.105263,0.0,0.0,0.000000,0.000000,0.026316,0.000000,0.078947,0.026316,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.026316,0.00,0.026316,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,"Moore Park , Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.105263,0.105263,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0

# Printing Top 5 Most Common Venues

In [45]:
num_top_venues = 5

for hood in c_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = c_toronto_grouped[c_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
             venue  freq
0      Pizza Place  0.11
1   Sandwich Place  0.08
2     Dessert Shop  0.08
3              Gym  0.05
4  Thai Restaurant  0.05


----Davisville North----
               venue  freq
0     Sandwich Place  0.17
1     Breakfast Spot  0.17
2              Hotel  0.17
3  Food & Drink Shop  0.17
4               Park  0.17


----Forest Hill North & West----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2                 Park  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1                 Park  0.33
2          Swim School  0.33
3  American Restaurant  0.00
4   Salon / Barbershop  0.00


----Moore Park , Summerhill East----
                 venue  freq
0           Playground   1.0
1  American Restaurant   0.0
2   Salon / Barbershop   0.0
3        Jewelry Store   0.0
4   Light Rail Station   0.0


----North Toronto We

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = c_toronto_grouped['Neighborhood']

for ind in np.arange(c_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(c_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Sushi Restaurant,Café,Coffee Shop,Thai Restaurant,Gym,Indian Restaurant
1,Davisville North,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Garden
2,Forest Hill North & West,Sushi Restaurant,Trail,Jewelry Store,Park,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
3,Lawrence Park,Bus Line,Park,Swim School,Garden,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Yoga Studio
4,"Moore Park , Summerhill East",Playground,Yoga Studio,Diner,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Garden,Fried Chicken Joint


# K-Means Clustering

In [48]:
from sklearn.cluster import KMeans
kclusters = 5

c_toronto_grouped_clustering = c_toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(c_toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 3, 4, 2, 0, 1, 0, 0], dtype=int32)

In [49]:
central_toronto_data.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

In [53]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

c_toronto_merged = central_toronto_data
c_toronto_merged = c_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

c_toronto_merged.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Swim School,Garden,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Yoga Studio
1,62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Pool,Yoga Studio,Diner,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Fried Chicken Joint
2,67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Garden
3,68,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,3,Sushi Restaurant,Trail,Jewelry Store,Park,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio
4,73,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Clothing Store,Yoga Studio,Health & Beauty Service,Gift Shop,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop


In [54]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(c_toronto_merged['Latitude'], c_toronto_merged['Longitude'], c_toronto_merged['Neighborhood'], c_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 0, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M4P,-79.390197,0,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Garden
4,M4R,-79.405678,0,Coffee Shop,Clothing Store,Yoga Studio,Health & Beauty Service,Gift Shop,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop
5,M5R,-79.405678,0,Café,Sandwich Place,Coffee Shop,American Restaurant,Cosmetics Shop,History Museum,Indian Restaurant,Liquor Store,Donut Shop,Middle Eastern Restaurant
6,M4S,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Sushi Restaurant,Café,Coffee Shop,Thai Restaurant,Gym,Indian Restaurant
8,M4V,-79.400049,0,Pub,Coffee Shop,American Restaurant,Supermarket,Restaurant,Liquor Store,Light Rail Station,Pizza Place,Sports Bar,Sushi Restaurant


In [56]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 1, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M5N,-79.416936,1,Garden,Pool,Yoga Studio,Diner,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Fried Chicken Joint


In [57]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 2, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M4T,-79.38316,2,Playground,Yoga Studio,Diner,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Gas Station,Garden,Fried Chicken Joint


In [58]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 3, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5P,-79.411307,3,Sushi Restaurant,Trail,Jewelry Store,Park,Fried Chicken Joint,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Yoga Studio


In [59]:
c_toronto_merged.loc[c_toronto_merged['Cluster Labels'] == 4, c_toronto_merged.columns[[1] + list(range(5, c_toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,-79.38879,4,Bus Line,Park,Swim School,Garden,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Yoga Studio
